In [69]:
import numpy as np
import re

If I get another slow algorithm, I will have to create the functions to give a reason for not fitting

In [70]:
def process_input(line):
    info = line.split()
    slots = re.findall('[#/?]+',info[0])
    values = [int(i) for i in info[1].split(',')]
    return slots, values

def process_input2(line):
    info = line.split()
    slot_string = info[0]
    val_string = info[1]
    for i in range(4):
        slot_string += '?' + info[0]
        val_string += ',' + info[1]
    slots = re.findall('[#/?]+',slot_string)
    values =[int(i) for i in val_string.split(',')]
    return slots,values

In [170]:
DEBUG = True

def debug(*args):
    if DEBUG:
        print(*args)

def fit_value2slot(slot,value):
    debug("CALL v2s", slot, value)
    n = len(slot)
    if n<value:
        debug("RESULT v2s", '0 fit')
        return 0    #doesn't fit
    if n==value:
        debug("RESULT v2s", '1 fit')
        return 1    #perfectly fits
    if '#' not in slot:
        debug("RESULT v2s", 'optional: ',  1 + (n-value))
        return 1 + (n-value)
    mandatory = np.argwhere([s=='#' for s in slot]).flatten()
    left,right = mandatory[0], mandatory[-1]
    mandatory_size = 1+(right-left)
    if mandatory_size > value:
        debug("RESULT v2s", 'm0 fit')
        return 0
    overflow = value-mandatory_size
    # print(n-right-1)
    L,R = min(left,overflow),min(n-right-1,overflow)
    # print(left,right,overflow)
    debug("RESULT v2s", 'full', 1 + (L+R-overflow))
    return 1 + (L+R-overflow)

def fit_values2slot_legacy(slot,values):
    debug("CALL vs2s", slot, values)
    V = len(values)
    value = values[0]
    if V == 1:
        return fit_value2slot(slot,value)
    n = len(slot)
    movement_range = n - (sum(values[1:])+(V-1)) # The first value cannot be placed in the latter half as those must be reserved for the other values and 1 padding per value (V-1)
    if movement_range < value:
        return 0 #There is not enough space for all values, ignoring mandatory tokens
    count = 0
    for shift in range((movement_range-value)+1):
        end = value+shift
        if shift!=0:
            if slot[shift-1] == '#':
                break #Stop looping over the free movement if the first value is to the right of a mandatory token
        if slot[value+shift] != '#': #don't fit first value here if followed by '#' as it must be followed by a '?' (or end of slot but that won't be reached here as len(values) would have been 1)
            new_slot = slot[end+1:]
            add_count = fit_values2slot(new_slot, values[1:])
            if add_count == 0 and '#' not in new_slot: #giving the others values less space will never increase the amount of arrangements
                break
            count +=  add_count
    debug("RESULT vs2s", slot, values, count)
    return count

def fit_values2slot(slot,values):
    debug("CALL vs2s", slot, values)
    V = len(values)
    value = values[0]
    if V == 1:
        return fit_value2slot(slot,value)
    n = len(slot)
    movement_range = n - (sum(values[1:])+(V-1)) # The first value cannot be placed in the latter half as those must be reserved for the other values and 1 padding per value (V-1)
    if movement_range < value:
        return 0 #There is not enough space for all values, ignoring mandatory tokens
    count = 0
    for shift in range((movement_range-value)+1):
        end = value+shift
        if shift!=0:
            if slot[shift-1] == '#':
                break #Stop looping over the free movement if the first value is to the right of a mandatory token
        if slot[value+shift] != '#': #don't fit first value here if followed by '#' as it must be followed by a '?' (or end of slot but that won't be reached here as len(values) would have been 1)
            new_slot = slot[end+1:]
            if sum([token == '#' for token in new_slot]) > sum(values[1:]): #not enough values to fill the mandatory tokens
                break
            add_count = fit_values2slot(new_slot, values[1:])
            if add_count == 0 and '#' not in new_slot: #giving the others values less space will never increase the amount of arrangements
                break
            
            count +=  add_count
    debug("RESULT vs2s", slot, values, count)
    return count

def fit_values2slots(slots,values):
    debug("CALL vs2ss", slots, values)
    V = len(values)
    N = len(slots)
    slot = slots[0]
    if N == 1 and V!=0:
        return fit_values2slot(slot,values)
    mandatory = ['#' in slot for slot in slots]
    M = sum(mandatory)
    if V == 0:
        return 1 if M==0 else 0#1 way to fit 0 values in N slots
    if M==V: #One mandatory slot per value means all values must go individually in the mandatory slots
        count = 1
        for i,value in zip(np.where(mandatory)[0].flatten(), values):
            prod_val = fit_value2slot(slots[i],value)
            if prod_val == 0:
                return 0 #It's a product so once one value is 0, the result will always be 0 and no further calculations are necessary
            count *= fit_value2slot(slots[i],value)
        return count
    max_space = sum([len(slt) for slt in slots]) + (N-1)
    max_req = sum(values) + (V-1)
    debug("space_comp", max_space, max_req)
    if max_space < max_req:
        debug('space_comp', 'no fit')
        return 0 #Super roug estimate but should eliminate some calculations
    #At this point, I have 1+ values and 2+ slots
    fit_length = -1 #I can add padding + value. The first value needs no padding so the -1 cancels it out
    M_left = M #The amount of mandatory values left after this slot
    if '#' not in slot:
        count = fit_values2slots(slots[1:],values)
    else:
        M_left -=1
        count = 0#Lol, what if I just skip this slot?
    
    n = len(slot)
    for v in range(V):
        fit_length += 1 + values[v]
        if fit_length > n:
            break #The first v values no longer fit in slot regardless of mandatory tokens
        if V-(v+1) < M_left: 
            break #There are V values of which (v-1) are fitted in slot, so V-(v-1) are left
        debug('test', v, slots)
        add_val = fit_values2slot(slot,values[:v+1])
        if v+1!=V: #If there are values left to fit
            prod_val = fit_values2slots(slots[1:],values[v+1:])
            add_val *= prod_val
        count += add_val
    return count

        

    


In [157]:
filename = 'input'
file = open(filename + '.txt')

slots_list = []
values_list = []

slots_list2 = []
values_list2 = []

p1 = 0
for line in file.readlines():
    slots, values = process_input(line)
    slots_list.append(slots)
    values_list.append(values)
    slots2, values2 = process_input2(line)
    slots_list2.append(slots2)
    values_list2.append(values2)

In [158]:
p2 = 0
i = 0
for slots,values in zip(slots_list2, values_list2):
    print(f"{i}/1000")
    ap2 = fit_values2slots(slots, values)
    p2 += ap2
    i+=1
print('part2:', p2)

1/1000
2/1000
3/1000
4/1000
5/1000
6/1000
7/1000
8/1000
9/1000
10/1000
11/1000
12/1000
13/1000
14/1000
15/1000
16/1000
17/1000
18/1000
19/1000
20/1000
21/1000
22/1000
23/1000
24/1000
25/1000
26/1000
27/1000
28/1000
29/1000
30/1000
31/1000
32/1000
33/1000
34/1000
35/1000
36/1000
37/1000
38/1000
39/1000
40/1000
41/1000
42/1000
43/1000
44/1000
45/1000
46/1000
47/1000
48/1000
49/1000
50/1000
51/1000
52/1000
53/1000
54/1000
55/1000
56/1000
57/1000
58/1000
59/1000
60/1000
61/1000
62/1000
63/1000
64/1000
65/1000
66/1000
67/1000
68/1000
69/1000
70/1000
71/1000
72/1000
73/1000
74/1000
75/1000
76/1000
77/1000
78/1000
79/1000
80/1000
81/1000
82/1000
83/1000
84/1000
85/1000
86/1000
87/1000
88/1000
89/1000
90/1000
91/1000
92/1000
93/1000
94/1000
95/1000
96/1000
97/1000
98/1000
99/1000
100/1000
101/1000
102/1000
103/1000
104/1000
105/1000
106/1000
107/1000
108/1000
109/1000
110/1000


KeyboardInterrupt: 

(['????????????#?#????????????????#?#????????????????#?#????????????????#?#????????????????#?#???'],
 [3, 2, 5, 3, 2, 5, 3, 2, 5, 3, 2, 5, 3, 2, 5])

In [154]:
slots_list2[0]

['???', '###????', '###????', '###????', '###????', '###']

In [73]:
def fit_value2slot(slot,value):
    n = len(slot)
    if n<value:
        return 0    #doesn't fit
    if n==value:
        return 1    #perfectly fits
    if '#' not in slot:
        return 1 + (n-value)
    mandatory = np.argwhere([s=='#' for s in slot]).flatten()
    left,right = mandatory[0], mandatory[-1]
    mandatory_size = 1+(right-left)
    if mandatory_size > value:
        return 0
    overflow = value-mandatory_size
    # print(n-right-1)
    L,R = min(left,overflow),min(n-right-1,overflow)
    # print(left,right,overflow)
    return 1 + (L+R-overflow)

def fit_value2slots(slots,value):
    # print(slots,value)
    N = len(slots)
    if N==0:
        return 0
    if N==1:
        return fit_value2slot(slots[0],value)
    mandatory = ['#' in slot for slot in slots]
    Nmand = sum(mandatory)
    if Nmand > 1:
        return 0    #Too many mandatory slots
    if Nmand == 1:
        I = np.argwhere(mandatory).flatten()[0]
        return fit_value2slot(slots[I],value)
    count = 0
    for slot in slots:
        n = len(slot)
        if n >= value:
            count += 1 + (n-value)
    return count
    
def cut_slots(slots, cut):
    N = len(slots)
    slot = slots[0]
    n = len(slot)
    if n<cut:
        print("Should not be reached! (cut_slots)")
        return None
    elif n<=cut+1:
        if N==1:
            return []
        else:
            return slots[1:]
    else:
        if N==1:
            return [slot[cut+1:]]
        else:
            return [slot[cut+1:]] + slots[1:]

    

def old_fit_values2slots(slots,values):
    N = len(slots)
    V = len(values)
    if N==0 or V==0:
        # print("Nothing to be done", slots, values)
        return 0
    value = values[0]
    if V==1:
        return fit_value2slots(slots,value)
    
    worth = True
    count = 0
    for s in range(N):
        slot = slots[s]
        n = len(slot)
        if value<=n:
            for i in range(n-value+1):
                misfit = False
                cut = value+i
                if cut < n: #if not, we are on the end of the slot so the following token is '.' 
                    if slot[cut] == '#':
                        misfit = True #If true, the next token is '#' so value should move up as it would force the '#' to '.' (impossible)
                if not misfit:
                    new_slots = cut_slots(slots[s:],cut)
                    # print('-sub_call',new_slots, values[1:])
                    add_count = old_fit_values2slots(new_slots,values[1:])
                    count += add_count
                    # print('--sub_result',new_slots, values[1:],add_count)
                    if add_count == 0:
                        # worth = False
                        # break
                        pass

                if slot[i] == '#':
                    worth = False #If the leftmost token is mandatory, I cannot move the leftmost value to the right
                    break
        else:
            pass
            # print(value, ' doesnt fit in ', slot)
        if '#' in slot:
            worth = False #If there are mandatory tokens in this slot, I cannot skip it so no need to look at further slots for the leftmost value
            break
    return count

In [166]:
filename = 'input'
file = open(filename + '.txt')

old_p1,p1 = 0,0
i = 0
I_list = []
for slots,values in zip(slots_list, values_list):
    ap1 = fit_values2slots(slots,values)
    ap1_old = old_fit_values2slots(slots,values)
    # print(slots, values, '->', ap1)
    if ap1 != ap1_old:
        print("wrong here!")
        print(slots, values,i)
        I_list.append(i)
    old_p1 += ap1_old
    p1 += ap1
    i+=1
print("part1: ",p1, old_p1)

part1:  7173 7173


In [167]:
len(I_list)

0

In [168]:
DEBUG = True

In [169]:
slots = ['????????????#?#????????????????#?#????????????????#?#????????????????#?#????????????????#?#???']
values = [3, 2, 5, 3, 2, 5, 3, 2, 5, 3, 2, 5, 3, 2, 5]
solution = fit_values2slots(slots,values)
old_solution = 0#old_fit_values2slots(slots,values)
print(solution,"should be", old_solution)

CALL vs2ss ['????????????#?#????????????????#?#????????????????#?#????????????????#?#????????????????#?#???'] [3, 2, 5, 3, 2, 5, 3, 2, 5, 3, 2, 5, 3, 2, 5]
CALL vs2s ????????????#?#????????????????#?#????????????????#?#????????????????#?#????????????????#?#??? [3, 2, 5, 3, 2, 5, 3, 2, 5, 3, 2, 5, 3, 2, 5]
CALL vs2s ????????#?#????????????????#?#????????????????#?#????????????????#?#????????????????#?#??? [2, 5, 3, 2, 5, 3, 2, 5, 3, 2, 5, 3, 2, 5]
CALL vs2s ?????#?#????????????????#?#????????????????#?#????????????????#?#????????????????#?#??? [5, 3, 2, 5, 3, 2, 5, 3, 2, 5, 3, 2, 5]
CALL vs2s #????????????????#?#????????????????#?#????????????????#?#????????????????#?#??? [3, 2, 5, 3, 2, 5, 3, 2, 5, 3, 2, 5]
CALL vs2s ?????????????#?#????????????????#?#????????????????#?#????????????????#?#??? [2, 5, 3, 2, 5, 3, 2, 5, 3, 2, 5]
CALL vs2s ??????????#?#????????????????#?#????????????????#?#????????????????#?#??? [5, 3, 2, 5, 3, 2, 5, 3, 2, 5]
CALL vs2s ????#?#????????????????#?#???????????

KeyboardInterrupt: 

In [165]:
len(slots[0]), sum(values) + len(values)-1

(94, 64)

In [148]:
I = I_list[0]
slots = slots_list[I]
values = values_list[I]
solution = fit_values2slots(slots,values)
old_solution = old_fit_values2slots(slots,values)
print(slots, values, '->', solution, old_solution)

['?????#????##', '??'] [4, 3] -> 3 3
